In [82]:
# Libraries to be used in the model
import pandas as pd
import numpy as py
import datetime as dt

# Plotiing libraries
import matplotlib.pyplot as plt

#Model Libraries
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV
from sklearn import metrics

from statistics import mean
from sklearn.metrics import mean_absolute_error

from sklearn.metrics import accuracy_score
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import cross_val_score

######################################3

# Get unique values per column
def print_unique_values_per_column(d):
    for c in d.columns:
        if (d[c].nunique() < 30):
            print(c, d[c].nunique(), d[c].dtype, d[c].unique())
        else:
            print(c, d[c].nunique(), d[c].dtype)

# Print missing values
def print_missing_values(d):
    missing_total = d.isnull().sum().sort_values(ascending=False)
    missing_percent = (d.isnull().sum()/d.isnull().count()).sort_values(ascending=False)
    missing = pd.concat([missing_total, missing_percent], axis=1, keys=['Total', 'Percent'])
    missing = missing[missing['Percent'] > 0]
    print(missing)

def get_columns_with_nulls(d):
    missing_total = d.isnull().sum().sort_values(ascending=False)
    missing_percent = (d.isnull().sum()/d.isnull().count()).sort_values(ascending=False)
    missing = pd.concat([missing_total, missing_percent], axis=1, keys=['Total', 'Percent'])
    missing = missing[missing['Percent'] > 0]
    return missing.index.values

# Custom accuracy evaluation
def accuracy_score(estimator, X, y):
    estimator.fit(X, y)
    y_pred = estimator.predict(X)
    accuracy = mean(1 - mean_absolute_error(y, y_pred)/y_pred)
    return accuracy

In [109]:
# Reading the data
test = pd.read_csv('test.csv')
train = pd.read_csv('train_data.csv')
train

,ofd_date,country_code,fc_codes,station_code,OFD,Slam,Earlies_Exp,Earlies_Rec,MNR_SNR_Exp,Rollover,Returns,R_Sideline,Sideline
0,2021-06-30,C,"F6, F8, F14, F17",D33,14594,14568,782,896,615,767,35,2,4
1,2021-06-30,C,"F6, F8, F9, F14, F17, F18",D37,12736,13111,655,823,211,29,17,2,1
2,2021-06-30,C,"F1, F4, F6, F7, F13, F15, F16",D34,14562,15651,1028,1910,225,35,47,3,1
3,2021-06-30,C,"F2, F6, F7, F10, F12, F13, F14, F15, F19",D45,11165,11467,514,769,56,39,29,0,1
4,2021-06-30,C,"F6, F8, F13, F14, F17",D50,10006,10423,399,842,52,60,65,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
11304,2021-02-01,D,"F35, F37, F38, F39, F40, F42, F43, F44, F46, F...",D60,16025,15768,487,383,62,69,195,15,1
11305,2021-02-01,D,"F35, F36, F37, F38, F39, F40, F42, F43, F44, F...",D54,18986,18861,318,446,83,100,107,248,1
11306,2021-02-01,D,"F35, F48, F52, F53, F54, F56",D69,12479,12345,163,196,64,64,62,136,0
11307,2021-02-01,D,"F35, F36, F40, F42, F43, F46, F48, F49, F52, F...",D59,16828,16504,248,333,83,135,112,256,1


In [110]:
# Data transformation
## Create the target column
train["Target"] = train['Earlies_Exp'] - train['MNR_SNR_Exp']

"""
Create the column of days of the week with the following coding
0 = Monday
1 = Tuesday
2 = Wednesday
3 = Thrusday
4 = Friday
5 = Saturday
6 = Sunday
""" 
train["ofd_date"] = pd.to_datetime(train["ofd_date"])

train["dayOdWeek"] = train['ofd_date'].dt.day_of_week
#train["dayOfWeekName"] = train['ofd_date'].dt.day_name()


In [114]:
#train['fc_codes'] = train['fc_codes'].str.split(',', n=79 , expand=True)
#pd.concat([train,train.fc_codes.str.get_dummies(sep=',')],1)

x= list(train['fc_codes'].split(','))
print(x)

['F1', ' F2', ' F4', ' F6', ' F7', ' F13', ' F15', ' F16', ' F19']


In [54]:
train

,ofd_date,country_code,fc_codes,station_code,OFD,Slam,Earlies_Exp,Earlies_Rec,MNR_SNR_Exp,Rollover,Returns,R_Sideline,Sideline,Target
0,Wednesday,C,"F6, F8, F14, F17",D33,14594,14568,782,896,615,767,35,2,4,167
1,Wednesday,C,"F6, F8, F9, F14, F17, F18",D37,12736,13111,655,823,211,29,17,2,1,444
2,Wednesday,C,"F1, F4, F6, F7, F13, F15, F16",D34,14562,15651,1028,1910,225,35,47,3,1,803
3,Wednesday,C,"F2, F6, F7, F10, F12, F13, F14, F15, F19",D45,11165,11467,514,769,56,39,29,0,1,458
4,Wednesday,C,"F6, F8, F13, F14, F17",D50,10006,10423,399,842,52,60,65,1,1,347
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11304,Monday,D,"F35, F37, F38, F39, F40, F42, F43, F44, F46, F...",D60,16025,15768,487,383,62,69,195,15,1,425
11305,Monday,D,"F35, F36, F37, F38, F39, F40, F42, F43, F44, F...",D54,18986,18861,318,446,83,100,107,248,1,235
11306,Monday,D,"F35, F48, F52, F53, F54, F56",D69,12479,12345,163,196,64,64,62,136,0,99
11307,Monday,D,"F35, F36, F40, F42, F43, F46, F48, F49, F52, F...",D59,16828,16504,248,333,83,135,112,256,1,165


In [12]:
#Check the ammount of null data
train.isna().sum()

ofd_date        0
country_code    0
fc_codes        0
station_code    0
OFD             0
Slam            0
Earlies_Exp     0
Earlies_Rec     0
MNR_SNR_Exp     0
Rollover        0
Returns         0
R_Sideline      0
Sideline        0
Target          0
dtype: int64

In [24]:
corr = train.corr()
corr["Target"].sort_values(ascending=False)

Target         1.000000
Earlies_Exp    0.755037
Earlies_Rec    0.400709
OFD            0.290058
Sideline       0.265911
Slam           0.245184
Returns        0.036127
R_Sideline     0.026542
Rollover      -0.024841
MNR_SNR_Exp   -0.552011
Name: Target, dtype: float64

In [21]:
#Drop no relevant columns
train = train.drop(columns=['ofd_date','fc_codes'])

In [22]:
#Get the categorical data to be separated
train = pd.get_dummies(train, prefix = ['Is_'], columns = ['country_code'])

In [23]:
# - Standarized the numerical data (if needed)
 for column in ['']:
    train[column] = (train[column] - train[column].mean()) / train[column].std()
train

,station_code,OFD,Slam,Earlies_Exp,Earlies_Rec,MNR_SNR_Exp,Rollover,Returns,R_Sideline,Sideline,Is__A,Is__B,Is__C,Is__D
0,D33,14594,14568,782,896,615,767,35,2,4,0,0,1,0
1,D37,12736,13111,655,823,211,29,17,2,1,0,0,1,0
2,D34,14562,15651,1028,1910,225,35,47,3,1,0,0,1,0
3,D45,11165,11467,514,769,56,39,29,0,1,0,0,1,0
4,D50,10006,10423,399,842,52,60,65,1,1,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11304,D60,16025,15768,487,383,62,69,195,15,1,0,0,0,1
11305,D54,18986,18861,318,446,83,100,107,248,1,0,0,0,1
11306,D69,12479,12345,163,196,64,64,62,136,0,0,0,0,1
11307,D59,16828,16504,248,333,83,135,112,256,1,0,0,0,1


In [24]:
#Preparing the variables for the model.

X = train.copy()
Y = train['station_code']
X = train.drop(['station_code'], axis=1)

In [25]:
#Model creation: Linear Regression 

X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=1)

LinearModel = LinearRegression()
LinearModel.fit(X_train, y_train)

ValueError: could not convert string to float: 'D17'

In [26]:
#Model creation: Logistic Regression

trainX, testX, trainY, testY = train_test_split(X, Y, test_size = 0.2, random_state=42)

LogisticsModel = LogisticRegression()


LogisticsModel.fit(trainX,trainY)
predY = LogisticsModel.predict(testX)

print(metrics.accuracy_score(testY, predY))
print(metrics.roc_auc_score(testY, predY))


0.048187444739168875


C:\Users\Mario Jesus\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\linear_model\_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


ValueError: could not convert string to float: 'D3'

In [ ]:
#Model creation: SVC

#   # define the parameters grid  
params = [{'C': [1, 2], 'kernel': ['poly'], 'degree': [1, 2]}]
#   # get the models calling GridSearchCV
models = GridSearchCV(SVC(), params, scoring='roc_auc', cv=4)
#   # fit the models
models.fit(X, Y)
#   # At this point you have available in models all the attributes described for GridSearchCV

print(models.best_params_)
print(models.best_score_)